In [2]:
import numpy as np
import cv2 as cv
import os
sift_all = []
for i in range(len(os.listdir('imgs'))):
    img = cv.imread('imgs/'+os.listdir('imgs')[i])
    sift = cv.SIFT_create()
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    kp = []
    des = []
    k = 60
    for l in range(img.shape[0]//k):
        for j in range(img.shape[1]//k):
            kp_block = sift.detect(gray[l*k:(l+1)*k,j*k:(j+1)*k], None)
            # extract only the strongest keypoints in each block
            kp_block = sorted(kp_block, key=lambda x: -x.response)[:1]
            # if there is no keypoint in the block, take the centre point as the keypoint
            if len(kp_block) == 0:
                kp_block = [cv.KeyPoint(l*k+k/2,j*k+k/2,k)]
            kp_block,des_block = sift.compute(gray[l*k:(l+1)*k,j*k:(j+1)*k],kp_block)

            kp.append(kp_block)
            des.append(des_block)
    
    sift_all.extend(des)
sift_all = np.array(sift_all)
sift_all = sift_all.reshape(sift_all.shape[0],sift_all.shape[2])

In [3]:
def distance(x,y):
    return np.sqrt(np.sum((x-y)**2))

def mean(x):
    return np.sum(x,axis=0)/len(x)

def k_means_clustering(X,k):
    # centers = X[np.random.choice(k,X.shape[1],replace=False),:]
    # initializing centers to 0
    centers = np.zeros((k,X.shape[1]))

    # randomly initializing centers
    for i in range(k):
        centers[i] = X[np.random.randint(0,X.shape[0]),:]

    labels = np.zeros(X.shape[0])
    
    dist = np.zeros((X.shape[0],k))
 
    n_iter = 0
    
    n_changes = 0

    prev_labels = centers.copy()
    while True:
        for i in range(k):
            # dist[:,i] = np.apply_along_axis(distance,1,X,centers[i,:])
            dist[:, i] = np.sqrt(np.sum((X - centers[i, :]) ** 2, axis=1))
        labels = np.argmin(dist,axis=1)
       
        for i in range(k):
            if(len(X[labels==i])>0):
                centers[i,:] = mean(X[labels==i,:])
  
        n_changes = len(centers[np.abs(centers-prev_labels)>0.0001*np.ones(centers.shape)])
    
        prev_labels = centers.copy()
       
        n_iter += 1
  
        if n_changes==0:
            break
    return labels,centers,n_iter

In [5]:
labels,centers,n_iter = k_means_clustering(sift_all,30)

In [ ]:
with open('image_words_sift.txt','w') as f:
    for i in range(centers.shape[0]):
        f.write(str(centers[i,:])+'\n')